In [1]:
#CAUTION: Takes a long time to load.
install.packages("themis")
devtools::install_github("tidymodels/tune")
install.packages("kknn")

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)

Skipping install of 'tune' from a github remote, the SHA1 (597ddf7a) has not changed since last install.
  Use `force = TRUE` to force installation

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



In [2]:
#load libraries
library(tidyverse)
library(repr)
library(tidymodels)
library(themis)
options(repr.matrix.max.rows = 10)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.4          ✔ rsample      1.1.1     
✔ dials        1.2.0          ✔ tune         1.1.1.9000
✔ infer        1.0.4          ✔ workflows    1.1.3     
✔ modeldata    1.1.0          ✔ workflowsets 1.0.1     
✔ parsnip      1.1.0          ✔ yardstick    1.1.0     
✔ recipes   

In [3]:
#Add columns
unscaled_data <- read_csv("pulsar_data.csv", 
                        col_names = c("mean_integrated_profile", 
                                      "stand_dev_integrated_profile", 
                                      "exc_kurtosis_integrated_profile", 
                                      "skew_integrated_profile",
                                      "mean_dmsnr", 
                                      "stand_dev_dmsnr", 
                                      "exc_kurtosis_dmsnr", 
                                      "skew_dmsnr", "class")) 

unscaled_data <- unscaled_data |>
    mutate(class = as_factor(class))
#unscaled_data 

#Class proportions in unscaled dataset (Imbalanced)
num_obs <- nrow(unscaled_data)
pulsar_proportions <- unscaled_data |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(unscaled_data))
pulsar_proportions

#Scale data
#pulsar_recipe <- recipe(class ~ exc_kurtosis_integrated_profile + skew_dmsnr + stand_dev_integrated_profile + 
#                        mean_dmsnr + skew_integrated_profile + exc_kurtosis_dmsnr, data = unscaled_data) |>
#    step_scale(all_predictors()) |>
#    step_center(all_predictors()) |>
#    prep()

#standardized_pulsar <- bake(pulsar_recipe, unscaled_data)
#standardized_pulsar

Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): mean_integrated_profile, stand_dev_integrated_profile, exc_kurtosis...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


class,n,percent
<fct>,<int>,<dbl>
0,16259,90.842552
1,1639,9.157448


In [4]:
set.seed(1)
pulsar_split <- initial_split(unscaled_data, prop = 0.75, strata = class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split) 

#Check class proportions in the training set
pulsar_proportions <- pulsar_train |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(pulsar_train)) 
pulsar_proportions

#Standardize and Upsample data to balance the training set
pulsar_recipe <- recipe(class ~ ., data = pulsar_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    themis::step_upsample(class, over_ratio = 1, skip = FALSE) |>
    prep()
pulsar_recipe

pulsar_train <- bake(pulsar_recipe, pulsar_train)
pulsar_train

#Check new proportions in training set
new_pulsar_proportions <- pulsar_train |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(pulsar_train))
new_pulsar_proportions

dummy_recipe <- recipe(class ~ ., data = pulsar_train)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
    set_engine("kknn") |>
    set_mode("classification")

pulsar_fit <- workflow() |>
    add_recipe(dummy_recipe) |>
    add_model(knn_spec) |>
    fit(data = pulsar_train)
pulsar_fit

pulsar_test_predictions <- predict(pulsar_fit, pulsar_test) |>
    bind_cols(pulsar_test)
pulsar_test_predictions

pulsar_prediction_accuracy <- pulsar_test_predictions |>
    metrics(truth = class, estimate = .pred_class) |>
    filter(.metric == "accuracy")
pulsar_prediction_accuracy

confusion <- pulsar_test_predictions |>
    conf_mat(truth = class, estimate = .pred_class)
confusion

class,n,percent
<fct>,<int>,<dbl>
0,12200,90.888773
1,1223,9.111227




── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 8



── Training information 

Training data contained 13423 data points and no incomplete rows.



── Operations 

• Scaling for: mean_integrated_profile, ... | Trained

• Centering for: mean_integrated_profile, ... | Trained

• Up-sampling based on: class | Trained



mean_integrated_profile,stand_dev_integrated_profile,exc_kurtosis_integrated_profile,skew_integrated_profile,mean_dmsnr,stand_dev_dmsnr,exc_kurtosis_dmsnr,skew_dmsnr,class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
-0.3382028,1.80673446,-0.009077165,-0.3682483,-0.3705891,-0.5859249,0.5078324,0.2171164,0
-0.3182986,-1.05924615,-0.143049363,-0.1145006,-0.3210346,-0.2330929,-0.1273024,-0.3951781,0
1.0039566,1.55681725,-0.512672218,-0.3878759,-0.3031295,-0.2733441,-0.3149639,-0.4864979,0
-0.6885178,0.01970171,0.053749649,-0.2172864,-0.3719951,-0.6020723,0.5179523,0.2552567,0
0.3272118,0.32285369,-0.418436890,-0.3029714,-0.3938600,-0.8719274,2.4373284,3.5765731,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
-3.00874182,-2.1534919,3.8811359,3.9084502,0.8392652,1.9072175,-1.4380657,-0.9719605,1
-3.22064561,-0.4111971,3.0779063,1.8956762,2.0203681,2.3648486,-1.7075928,-1.0024231,1
-0.99198116,-0.2663260,0.7525602,0.1399375,0.3947089,1.3811141,-1.3643983,-0.9603344,1


class,n,percent
<fct>,<int>,<dbl>
0,12200,50
1,12200,50


══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: nearest_neighbor()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
kknn::train.kknn(formula = ..y ~ ., data = data, ks = min_rows(3,     data, 5), kernel = ~"rectangular")

Type of response variable: nominal
Minimal misclassification: 0.01385246
Best kernel: rectangular
Best k: 3

.pred_class,mean_integrated_profile,stand_dev_integrated_profile,exc_kurtosis_integrated_profile,skew_integrated_profile,mean_dmsnr,stand_dev_dmsnr,exc_kurtosis_dmsnr,skew_dmsnr,class
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0,140.56250,55.68378,-0.23457141,-0.6996484,3.199833,19.11043,7.975532,74.24222,0
0,88.72656,40.67223,0.60086608,1.1234917,1.178930,11.46872,14.269573,252.56731,0
0,142.07812,45.28807,-0.32032843,0.2839525,5.376254,29.00990,6.076266,37.83139,0
0,133.25781,44.05824,-0.08105986,0.1153615,1.632107,12.00781,11.972067,195.54345,0
0,109.64062,49.01765,0.13763583,-0.2566998,1.508361,12.07290,13.367926,223.43842,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0,119.00781,52.09104,0.21529353,-0.02022867,2.0125418,18.754594,10.339768,111.76052,0
0,133.14062,43.70618,-0.01268608,0.23239435,2.0919732,13.489370,9.821958,131.52103,0
0,90.14844,40.78604,0.45713856,0.88551621,5.3545150,29.759511,6.102231,37.97952,0


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.907486


          Truth
Prediction    0    1
         0 4059  414
         1    0    2

In [5]:
#10 fold cross validation
pulsar_vfold <- vfold_cv(pulsar_train, v = 10, strata = class)

pulsar_resample_fit <- workflow() |>
       add_recipe(dummy_recipe) |>
       add_model(knn_spec) |>
       fit_resamples(resamples = pulsar_vfold)

pulsar_metrics <- collect_metrics(pulsar_resample_fit)
pulsar_metrics

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.9852459,10,0.0010421984,Preprocessor1_Model1
roc_auc,binary,0.9913005,10,0.0008092232,Preprocessor1_Model1


In [ ]:
#CAUTION: Takes a long time to load.
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")
#knn_tune

k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

knn_results <- workflow() |>
               add_recipe(dummy_recipe) |>
               add_model(knn_tune) |>
               tune_grid(resamples = pulsar_vfold, grid = k_vals) |>
               collect_metrics()
#knn_results

accuracies <- knn_results |> 
       filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate")  



accuracy_versus_k


**Selecting Predictors**

In [ ]:
names <- colnames(pulsar_train |> select(-class))
names

In [ ]:
example_formula <- paste("class", "~", paste(names, collapse="+"))
example_formula

In [ ]:
# create an empty tibble to store the results
predictor_accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

In [ ]:
# store the total number of predictors
n_total <- length(names)

# stores selected predictors
selected <- c()

# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("class", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        pulsar_recipe <- recipe(as.formula(model_string), 
                                data = pulsar_train) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())

        # tune the KNN classifier with these predictors, 
        # and collect the accuracy for the best K
        acc <- workflow() |>
          add_recipe(pulsar_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = pulsar_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
predictor_accuracies <- predictor_accuracies |> 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}
predictor_accuracies

Based on the data above, the highest accuracy is obtained by having exc_kurtosis_integrated_profile+skew_dmsnr+stand_dev_integrated_profile+mean_dmsnr as predictors